# Day 9: Smoke Basin
https://adventofcode.com/2021/day/9

In [1]:
import os
from dataclasses import dataclass, field
from functools import reduce
from operator import itemgetter, mul
from typing import Optional, List, Tuple

## Part 1

In [2]:
@dataclass(order=True, eq=True)
class SeaFloorSpace():
    row: int = field(default=0, compare=True)
    col: int = field(default=0, compare=True)
    height: int = field(default=0, compare=True)
    low_point: bool = field(default=False, compare=False)

class SeaFloor:
    rows: list[list[SeaFloorSpace]] 

    def __init__(self, input_file:str):
        self.rows = list()
        with open(input_file) as f:
            row = 0
            for line in f:
                if line.rstrip():
                    self.rows.append([SeaFloorSpace(row, col, int(x)) for col, x in enumerate(line.strip())])
                    row = row + 1

    def get_seafloor(self, row_idx, col_idx) -> Optional[SeaFloorSpace]:
        sf: Optional[SeaFloorSpace] = None
        if 0 <= row_idx < len(self.rows) and 0 <= col_idx < len(self.rows[row_idx]):
            sf = self.rows[row_idx][col_idx]
        return sf

    def find_low_points(self) -> None:
        """
        Marks low points in this Sea Floor
        """
        for row_idx, row in enumerate(self.rows):
            for col_idx, col in enumerate(row):
                is_low_point: bool = True
                is_low_point &= (  # Test to the left.
                    bool(row[col_idx].height < row[col_idx-1].height) 
                    if self.get_seafloor(row_idx, col_idx-1) 
                    else True
                )
                is_low_point &= (  # Test to the right.
                    bool(row[col_idx].height < row[col_idx+1].height) 
                    if self.get_seafloor(row_idx, col_idx+1) 
                    else True
                )
                is_low_point &= (  # Test above.
                    bool(row[col_idx].height < self.get_seafloor(row_idx-1, col_idx).height) 
                    if self.get_seafloor(row_idx-1, col_idx) 
                    else True
                )
                is_low_point &= (  # Test below.
                    bool(row[col_idx].height < self.get_seafloor(row_idx+1, col_idx).height) 
                    if self.get_seafloor(row_idx+1, col_idx) 
                    else True
                )
                if is_low_point:
                    self.rows[row_idx][col_idx].low_point = True
    
    def low_points(self) -> list:
        """
        Returns List of tuples for low points.
        Tuple(row_index, col_index, height)
        """
        self.find_low_points()  # double check
        return [
            (ridx,cidx, seafloorspace.height)
            for ridx, row in enumerate(self.rows)
            for cidx, seafloorspace in enumerate(row)
            if seafloorspace.low_point
        ]

    def risk_sum(self) -> int:
        """
        Returns sum of risk (height + 1) for low points.
        """
        self.find_low_points()  # double check
        return sum([
            seafloorspace.height + 1
            for ridx, row in enumerate(self.rows)
            for cidx, seafloorspace in enumerate(row)
            if seafloorspace.low_point
        ])

    def __repr__(self) -> str:
        value = ""
        for row in self.rows:
            value = value + "".join([str(col.height) for col in row]) + "\n"
        return value
    
    def basin_size_explore_recrusive(
        self, 
        row_idx:int, 
        col_idx:int, 
        basin_members:Optional[List[SeaFloorSpace]] = None
    ) -> int:
        """
        Recursive walk of the adjacent spaces not already placed into the Basin
        Parameters
            row_idx: <int> row index of "rows" where to search
            row_idx: <int> col index of "rows" where to search
            basin_members: is a list of Spaces already in the Basin to avoid
                infinite loops when exploring the space.
        Returns
            <int> Basin Size        
        """

        if basin_members is None:
            basin_members = list()

        if (not self.get_seafloor(row_idx, col_idx) 
            or self.get_seafloor(row_idx, col_idx).height == 9
            or self.get_seafloor(row_idx, col_idx) in basin_members
        ):
            # print(f"Reject ({row_idx}, {col_idx})")
            return 0
        else:
            basin_members.append(self.get_seafloor(row_idx, col_idx))
            # print(f"Add ({row_idx}, {col_idx})")
            return 1\
                + self.basin_size_explore_recrusive(row_idx-1, col_idx, basin_members)\
                + self.basin_size_explore_recrusive(row_idx+1, col_idx, basin_members)\
                + self.basin_size_explore_recrusive(row_idx, col_idx-1, basin_members)\
                + self.basin_size_explore_recrusive(row_idx, col_idx+1, basin_members)

    def largest_n_basins(self, n: int = 3) -> List[Tuple[int, int, int]]:
        """
        Returns a tuple of the N largest basins.
        Returns
            Tuple(low_point_row_index, low_point_col_index, size)
        """
        self.find_low_points()  # double check
        basins: List[Tuple[int, int, int]] = list()
        for ridx, cidx, _ in self.low_points():
            basin_size = self.basin_size_explore_recrusive(ridx, cidx)
            basins.append((ridx, cidx, basin_size))
        
        if n > len(basins):
            n = len(basins)

        return sorted(basins,key=itemgetter(2))[-1*n:]

# input data.
def test_input_location(
    file_loc: str = 'test_input.txt', 
    data_directory: str  = 'data/day_9'
) -> str:
    return os.path.join(data_directory, file_loc)

def input_location(
    file_loc: str = 'input.txt', 
    data_directory: str  = 'data/day_9'
) -> str:
    return os.path.join(data_directory, file_loc)

In [3]:
board = SeaFloor(test_input_location())
board.find_low_points()
assert board.risk_sum() == 15
board

2199943210
3987894921
9856789892
8767896789
9899965678

In [4]:
real_board = SeaFloor(input_location())
real_board.find_low_points()
real_board.risk_sum()

516

## Part 2

In [5]:
board.low_points()

[(0, 1, 1), (0, 9, 0), (2, 2, 5), (4, 6, 5)]

In [6]:
assert board.basin_size_explore_recrusive(0, 1) == 3
assert board.basin_size_explore_recrusive(0, 9) == 9
assert board.basin_size_explore_recrusive(2, 2) == 14
assert board.basin_size_explore_recrusive(4, 6) == 9
assert reduce(mul, [b[2] for b in board.largest_n_basins(n=3)]) == 1134

In [7]:
reduce(mul, [b[2] for b in real_board.largest_n_basins(n=3)])

1023660